In [65]:
import string, copy, re
from collections import deque, defaultdict
import numpy as np
from aoclib import *

inp = open("in24.txt").read().strip()

components = set(tuple(int(x) for x in line.split("/")) for line in inp.splitlines())

## Slow solution
def plug_in_next(chain):
    chains = []
    for c in components:
        if frozenset(c) in set((frozenset(x) for x in chain)):
            continue     
        if chain[-1][1] in c:
            if c[0] == chain[-1][1]:
                chains.append(chain + [ [c[0], c[1]] ])
            else:
                chains.append(chain + [ [c[1], c[0]] ])
    if chains:
        return chains
    else:
        return None

# Pseudo-component as a starting anchor
chains = [[[0, 0]]]
finished_chains = []
while chains:
    new_chains = []
    for chain in chains:
        grown_chains = plug_in_next(chain)
        if grown_chains:
            new_chains.extend(grown_chains)
        else:
            finished_chains.append(chain)
    chains = new_chains

# Part 1
scores = [sum([x + y for x, y in chain]) for ch in finished_chains]
print("Part 1:", max(scores))
# Part 2
max_len = max(len(x) for x in finished_chains)
longest_chains = filter(lambda chain: len(chain) == max_len, finished_chains)
scores = map(lambda chain: sum([x + y for x, y in chain]), longest_chains)
print("Part 2:", max(scores))

1868

In [89]:
## Fast solution by u794575248

data = []
with open('in24.txt', 'r') as f:
    for line in f.readlines():
        a, b = line.split('/')
        data.append((int(a), int(b)))

bridge = ([], 0)

def run(b, d):
    available = [a for a in d if b[1] in a]
    if len(available) == 0:
        yield b
    else:
        for a in available:
            d_ = d.copy()
            d_.remove(a)
            for q in run((b[0] + [a], a[0] if b[1] == a[1] else a[1]), d_):
                yield q

# part 1
x = list(map(lambda bridge: sum([a + b for a, b in bridge[0]]), run(bridge, data)))
print(max(x))
# part 2
max_len = max(map(lambda bridge: len(bridge[0]), run(bridge, data)))
long = filter(lambda bridge: len(bridge[0]) == max_len, run(bridge, data))
print(max(map(lambda bridge: sum([a + b for a, b in bridge[0]]), long)))


1868
1841


In [91]:
# Fast solution by mkeeter
def gen_bridges(bridge, components):
    bridge = bridge or [(0, 0)]
    cur = bridge[-1][1]
    for b in components[cur]:
        if not ((cur, b) in bridge or (b, cur) in bridge):
            new = bridge+[(cur, b)]
            yield new
            yield from gen_bridges(new, components)

def parse_components(input):
    components = defaultdict(set)
    for l in input.strip().splitlines():
        a, b = [int(x) for x in l.split('/')]
        components[a].add(b)
        components[b].add(a)
    return components

def solve(input):
    components = parse_components(input)
    mx = []
    for bridge in gen_bridges(None, components):
        mx.append((len(bridge), sum(a+b for a, b in bridge)))
    return mx

solution = solve(inp)
part1 = sorted(solution, key=lambda x: x[1])[-1][1]
part2 = sorted(solution)[-1][1]
print(part1)
print(part2)

1868
1841
